In [1]:
require 'hdf5'
require 'nn'

In [2]:
-- Loading data
myFile = hdf5.open('../data/MM_data_cap.hdf5','r')
data = myFile:all()
input_matrix_train_cap = data['input_matrix_train_cap']
input_matrix_dev_cap = data['input_matrix_dev_cap']
input_matrix_test_cap = data['input_matrix_test_cap']
myFile:close()

In [17]:
-- Loading data
myFile = hdf5.open('../data/sent_start.hdf5','r')
data = myFile:all()
sent = data['sent_start']
myFile:close()

In [8]:
nwords = input_matrix_train_cap:size(1)
train_input = torch.Tensor(nwords-1,10)
train_input:narrow(2,1,1):copy(input_matrix_train_cap:narrow(2,1,1):narrow(1,2,nwords-1))
train_input:narrow(2,2,9):copy(input_matrix_train_cap:narrow(2,2,9):narrow(1,1,nwords-1))
train_output = input_matrix_train_cap:narrow(2,16,1):narrow(1,2,nwords-1)

In [11]:
myFile = hdf5.open('../data/embeddings.hdf5','r')
data2 = myFile:all()
embeddings = data2['embeddings']
myFile:close()

In [13]:
LT = nn.LookupTable(400002,50)
LT.weight:narrow(1, 1, 400000):copy(embeddings)

In [15]:
model = nn.Sequential()
t1 = nn.ParallelTable()

t1_1 = nn.Sequential()
t1_1:add(LT)
t1_1:add(nn.View(-1,50))

t1_2 = nn.Identity()

t1:add(t1_1)
t1:add(t1_2)

model:add(t1)
model:add(nn.JoinTable(2))

model:add(nn.Linear(59,9))

In [80]:
function compute_logscore(observations, i, model, C)
    local y = torch.zeros(C,C)
    local hot_1 = torch.zeros(C)
    for j = 1, C do
        hot_1:zero()
        hot_1[j] = 1
        y:narrow(1,j,1):copy(model:forward({observations[i]:view(1,1),hot_1:view(1,9)}))
    end
    return y
end

function viterbi(observations, compute_logscore, model, C)
    
    local y = torch.zeros(C,C)
    -- Formating tensors
    local initial = torch.zeros(C, 1)
    -- initial started with a start of sentence: <t>

    initial[{8,1}] = 1
    initial:log()

    -- number of classes
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = (initial + compute_logscore(observations, 1, model, C)[8]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
       
        y:copy(compute_logscore(observations, i, model, C))
        scores = y:transpose(1,2) + maxes:view(1, C):expand(C, C)

        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)

        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
    end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end

    return classes
end

function train_model(train_input, sent, train_output, model, din, nclass, eta, nEpochs)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01
    loss = torch.Tensor(nEpochs)

    -- To store the loss
    av_L = 0

    -- Memory allocation
    inputs_batch = torch.DoubleTensor(100, din)
    gold_sequence = torch.DoubleTensor(100)
    high_score_seq = torch.DoubleTensor(100)
    df_do = torch.zeros(100, nclass)

    for i = 1, nEpochs do
        -- timing the epoch
        timer = torch.Timer()
        av_L = 0
        
        -- mini batch loop
        for t = 2, sent:size(1) do
            -- Mini batch data
            sent_size = sent[{t,2}]
--             print('here1')
            
            inputs_batch:narrow(1,1,sent_size):copy(train_input:narrow(1,sent[{t,1}]-1,sent_size))
--             print('here2')
            
            gold_sequence:narrow(1,1,sent_size):copy(train_output:narrow(1,sent[{t,1}]-1,sent_size))
--             print('here3')
            
            -- reset gradients
            model:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass on a batch subsequence:
            high_score_seq:narrow(1,1,sent_size):copy(viterbi(inputs_batch:narrow(1,1,sent_size):narrow(2,1,1), 
                                                                compute_logscore, model, nclass))
--             print('here4')
            
            -- TO CODE :
            df_do:zero()
            for ii = 1, sent_size do
                if high_score_seq[ii] ~= gold_sequence[ii] then
                    y = model:forward({inputs_batch:narrow(1,ii,1):narrow(2,1,1),inputs_batch:narrow(1,ii,1):narrow(2,2,9)})
                    df_do[ii]:copy(y)
                end
            end
--             print('here6')
            model:backward({inputs_batch:narrow(1,1,sent_size):narrow(2,1,1), inputs_batch:narrow(1,1,sent_size):narrow(2,2,9)}, 
                            df_do:narrow(1,1,sent_size))
--             print('here7')
            model:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        loss[i] = av_L/math.floor(train_input:size(1)/batchSize)
        print('Average Loss: '.. loss[i])
       
    end
    
    return loss
end

In [81]:
train_model(train_input, sent, train_output, model, 10, 9, 0.001, 1)

...virgileaudi/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 3 module of nn.Sequential:
...rs/virgileaudi/torch/install/share/lua/5.1/nn/Linear.lua:80: size mismatch, t: [1 x 59], m1: [4 x 9], m2: [9 x 59] at /tmp/luarocks_torch-scm-1-7765/torch7/lib/TH/generic/THTensorMath.c:780
stack traceback:
	[C]: in function 'addmm'
	...rs/virgileaudi/torch/install/share/lua/5.1/nn/Linear.lua:80: in function 'updateGradInput'
	...rs/virgileaudi/torch/install/share/lua/5.1/nn/Module.lua:31: in function <...rs/virgileaudi/torch/install/share/lua/5.1/nn/Module.lua:29>
	[C]: in function 'xpcall'
	...virgileaudi/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...irgileaudi/torch/install/share/lua/5.1/nn/Sequential.lua:84: in function 'backward'
	[string "function compute_logscore(observations, i, mo..."]:107: in function 'f'
	[string "local f = function() return train_model(train..."]:1: in main chunk
	[C]: in function 'xpcall'
	.../virgileaudi/torch/install/share/

In [82]:
inputs_batch = torch.DoubleTensor(100, 10)
gold_sequence = torch.DoubleTensor(100)
high_score_seq = torch.DoubleTensor(100)
df_do = torch.zeros(100, 9)

In [83]:
t = 2
sent_size = sent[{t,2}]
--             print('here1')

inputs_batch:narrow(1,1,sent_size):copy(train_input:narrow(1,sent[{t,1}]-1,sent_size))
--             print('here2')

gold_sequence:narrow(1,1,sent_size):copy(train_output:narrow(1,sent[{t,1}]-1,sent_size))
--             print('here3')

In [84]:
high_score_seq:narrow(1,1,sent_size):copy(viterbi(inputs_batch:narrow(1,1,sent_size):narrow(2,1,1), compute_logscore, model, 9));

In [109]:
for ii = 1, sent_size do
    if high_score_seq[ii] ~= gold_sequence[ii] then
        print('yes')
        pr = model:forward({inputs_batch:narrow(1,ii,1):narrow(2,1,1),inputs_batch:narrow(1,ii,1):narrow(2,2,9)})
        m, a = pr:max(2)
        print(ii, a[1][1])
        df_do[{ii,a[1][1]}] = 1
        df_do[{ii,gold_sequence[ii]}] = -1          
    else
        print('no')
    end
    
end

no	
yes	
2	8	
yes	
3	4	
yes	


4	2	


In [110]:
df_do:narrow(1,1,sent_size)

 0  1  0  0  0  0  0  0  0
 1 -1  1  1  1  1  1  1  1
 1 -1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1 -1
[torch.DoubleTensor of size 4x9]



In [107]:
sent[{t,1}]-1

11	
